# Обучение сверточной сети на практике

## Загрузка датасета из Kaggle

In [1]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle

In [3]:
!cp /content/drive/MyDrive/Kaggle_API/kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!make

pip install kaggle
mkdir ~/.kaggle
mkdir: cannot create directory ‘/root/.kaggle’: File exists
make: *** [Makefile:7: dir] Error 1


In [5]:
import kaggle

In [6]:
kaggle.api.authenticate()

In [7]:
kaggle.api.competition_download_files('dogs-vs-cats-redux-kernels-edition', path='datas')